In [2]:
import requests
from lxml import html
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import random as rd
import time

In [3]:
headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 Edg/109.0.1518.78',
    'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'
}
response = requests.get('https://statsroyale.com/cards', headers = headers)
soup = bs(response.text, "html.parser")

CardIndex = {}
data = soup.select('.cards__group')

In [4]:
#獲取卡片名稱，順便取得圖像
Party_Cards = ['Terry', 'Super Lava Hound', 'Super Witch', 'Santa Hog Rider', 'Super Magic Archer', 'Super Mini P.E.K.K.A', 'Super Ice Golem', 'Super Archers', 'Party Hut', 'Party Rocket', 'Warmth']

for i in data:
    Card_type = i.find('div',attrs={"class" : "ui__headerSmall"}).text
    CardIndex[Card_type] = []
    for j in i.find_all('div', attrs ={'class' : 'cards__card'}):
        name = j.find('div', attrs ={'class' : 'ui__tooltip'}).text.strip()
        Elixir = j['data-elixir']
        if name[0] == ' ' :name = name[1:]
        if name[-1] == ' ' :name = name[:-1]
        if name in Party_Cards:continue

        CardIndex[Card_type].append([name, Elixir])

        image_url = j.find('img')['src']
        image_data = requests.get(image_url).content
        with open('../data/Cards/Cards Image/' + name + '.png', 'wb') as image:
            image.write(image_data)
        time.sleep(rd.randint(5, 8))

In [5]:
#抓取14等的資料
#部分網站不完整 需要另外處理
Cards = []
need_skip = ['Rage', 'Heal Spirit', 'Mirror']
index = 0
for i in CardIndex.keys():
    for j in CardIndex[i]:
        if j[0] in need_skip:continue
        link = j[0].replace(' ', '+')
        response = requests.get('https://statsroyale.com/card/' + link, headers = headers)
        page = bs(response.text, "html.parser")
        Rarity = page.find('div', attrs = {'class' : 'card__count'}).text
        if '\n' in Rarity:Rarity = Rarity[:-1]
        Cards.append([i, j[0], Rarity, j[1]])
        time.sleep(rd.randint(5, 8))

        print(Cards[-1])
        
Cards.append(['Spell', 'Rage', 'Epic', '2'])
Cards.append(['Spell', 'Mirror', 'Epic', '1'])
Cards.append(['Troop', 'Heal Spirit', 'Common', '1'])

columns = ['Type', 'Name', 'Rarity', 'Elixir']
df = pd.DataFrame(Cards, columns = columns)[['Type', 'Rarity', 'Name', 'Elixir']]
df

['Troop', 'Knight', 'Common', '3']
['Troop', 'Archers', 'Common', '3']
['Troop', 'Goblins', 'Common', '2']
['Troop', 'Giant', 'Rare', '5']
['Troop', 'P.E.K.K.A', 'Epic', '7']
['Troop', 'Minions', 'Common', '3']
['Troop', 'Balloon', 'Epic', '5']
['Troop', 'Witch', 'Epic', '5']
['Troop', 'Barbarians', 'Common', '5']
['Troop', 'Golem', 'Epic', '8']
['Troop', 'Skeletons', 'Common', '1']
['Troop', 'Valkyrie', 'Rare', '4']
['Troop', 'Skeleton Army', 'Epic', '3']
['Troop', 'Bomber', 'Common', '2']
['Troop', 'Musketeer', 'Rare', '4']
['Troop', 'Baby Dragon', 'Epic', '4']
['Troop', 'Prince', 'Epic', '5']
['Troop', 'Wizard', 'Rare', '5']
['Troop', 'Mini P.E.K.K.A', 'Rare', '4']
['Troop', 'Spear Goblins', 'Common', '2']
['Troop', 'Giant Skeleton', 'Epic', '6']
['Troop', 'Hog Rider', 'Rare', '4']
['Troop', 'Minion Horde', 'Common', '5']
['Troop', 'Ice Wizard', 'Legendary', '3']
['Troop', 'Royal Giant', 'Common', '6']
['Troop', 'Guards', 'Epic', '3']
['Troop', 'Princess', 'Legendary', '3']
['Troop'

,Type,Rarity,Name,Elixir
0,Troop,Common,Knight,3
1,Troop,Common,Archers,3
2,Troop,Common,Goblins,2
3,Troop,Rare,Giant,5
4,Troop,Epic,P.E.K.K.A,7
...,...,...,...,...
105,Spell,Common,Giant Snowball,2
106,Spell,Common,Royal Delivery,3
107,Spell,Epic,Rage,2
108,Spell,Epic,Mirror,1


In [6]:
#下載進化卡圖片
from lxml import html

page = requests.get('https://www.deckshop.pro/card/list')
tree = html.fromstring(page.content)

pages = tree.xpath('//h3[text()="With evolutions"]/../div/a/img')
for content in pages:
    Evol_Name = 'Evol ' + content.attrib['alt']
    image_url = 'https://www.deckshop.pro' + content.attrib['src']

    image_data = requests.get(image_url).content
    with open('../data/Cards/Cards Image/' + Evol_Name + '.png', 'wb') as image:
        image.write(image_data)

    origin = df.loc[df['Name'] == content.attrib['alt']].values[0]
    df1 = pd.DataFrame([[origin[0], Evol_Name, 'Evolution', origin[3]]], columns = columns)
    df = pd.concat([df, df1])
    print(df.iloc[-1])

    time.sleep(rd.randint(5, 8))

Type               Troop
Rarity         Evolution
Name      Evol Skeletons
Elixir                 1
Name: 0, dtype: object
Type                Troop
Rarity          Evolution
Name      Evol Ice Spirit
Elixir                  1
Name: 0, dtype: object
Type            Troop
Rarity      Evolution
Name      Evol Bomber
Elixir              2
Name: 0, dtype: object
Type          Troop
Rarity    Evolution
Name      Evol Bats
Elixir            2
Name: 0, dtype: object
Type             Troop
Rarity       Evolution
Name      Evol Archers
Elixir               3
Name: 0, dtype: object
Type            Troop
Rarity      Evolution
Name      Evol Knight
Elixir              3
Name: 0, dtype: object
Type                 Troop
Rarity           Evolution
Name      Evol Firecracker
Elixir                   3
Name: 0, dtype: object
Type         Building
Rarity      Evolution
Name      Evol Mortar
Elixir              4
Name: 0, dtype: object
Type        Building
Rarity     Evolution
Name      Evol Tesla
Elixi

In [7]:
df

,Type,Rarity,Name,Elixir
0,Troop,Common,Knight,3
1,Troop,Common,Archers,3
2,Troop,Common,Goblins,2
3,Troop,Rare,Giant,5
4,Troop,Epic,P.E.K.K.A,7
...,...,...,...,...
0,Troop,Evolution,Evol Barbarians,5
0,Troop,Evolution,Evol Royal Giant,6
0,Troop,Evolution,Evol Royal Recruits,7
0,Troop,Evolution,Evol Valkyrie,4


In [8]:
from pandas.api.types import CategoricalDtype

type_order = CategoricalDtype(['Troop', 'Building', 'Spell'], ordered=True)
Rarity_order = CategoricalDtype(['Evolution', 'Champion', 'Legendary', 'Epic', 'Rare', 'Common'], ordered=True)
df['Type'] = df['Type'].astype(type_order)
df['Rarity'] = df['Rarity'].astype(Rarity_order)

df = df.sort_values(by = ['Rarity', 'Type', 'Elixir', 'Name'], ascending=[True, True, False, True]).drop_duplicates()
df

,Type,Rarity,Name,Elixir
0,Troop,Evolution,Evol Royal Recruits,7
0,Troop,Evolution,Evol Royal Giant,6
0,Troop,Evolution,Evol Barbarians,5
0,Troop,Evolution,Evol Valkyrie,4
0,Troop,Evolution,Evol Archers,3
...,...,...,...,...
78,Building,Common,Cannon,3
92,Spell,Common,Arrows,3
106,Spell,Common,Royal Delivery,3
105,Spell,Common,Giant Snowball,2


In [9]:
df.to_csv('../data/Cards/cards.csv', index = False)